In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Schizophrenia"
cohort = "GSE120342"

# Input paths
in_trait_dir = "../../input/GEO/Schizophrenia"
in_cohort_dir = "../../input/GEO/Schizophrenia/GSE120342"

# Output paths
out_data_file = "../../output/preprocess/Schizophrenia/GSE120342.csv"
out_gene_data_file = "../../output/preprocess/Schizophrenia/gene_data/GSE120342.csv"
out_clinical_data_file = "../../output/preprocess/Schizophrenia/clinical_data/GSE120342.csv"
json_path = "../../output/preprocess/Schizophrenia/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Aberrant transcriptomes and DNA methylomes define pathways that drive pathogenesis and loss of brain laterality/asymmetry in schizophrenia and bipolar disorder"
!Series_summary	"This SuperSeries is composed of the SubSeries listed below."
!Series_overall_design	"Refer to individual Series"
Sample Characteristics Dictionary:
{0: ['disease state: control', 'disease state: SCZ', 'disease state: BD(-)', 'disease state: BD(+)'], 1: ['laterality: left', 'laterality: right']}


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
import pandas as pd
import os
import json
from typing import Callable, Optional, Dict, Any

# 1. Gene Expression Data Availability
# Based on the background information, this appears to be a study about transcriptomes and DNA methylomes
# in schizophrenia and bipolar disorder, so gene expression data is likely available
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability
# Trait (Schizophrenia) data is in row 0 (disease state)
trait_row = 0
# There's no age information in the sample characteristics
age_row = None
# There's no gender information in the sample characteristics
gender_row = None

# 2.2 Data Type Conversion Functions
def convert_trait(value):
    """Convert trait value to binary (0 for control, 1 for SCZ)"""
    if value is None:
        return None
    
    # Extract the value part after colon if present
    if ':' in str(value):
        value = str(value).split(':', 1)[1].strip()
    
    value = str(value).lower()
    if 'control' in value:
        return 0
    elif 'scz' in value or 'schizophrenia' in value:
        return 1
    else:
        # This handles BD cases which aren't relevant for our Schizophrenia trait
        return None

def convert_age(value):
    """Convert age value to continuous (not used in this dataset)"""
    # This function is not actually used since age data is not available
    return None

def convert_gender(value):
    """Convert gender value to binary (not used in this dataset)"""
    # This function is not actually used since gender data is not available
    return None

# 3. Save Metadata
# Trait data is available since trait_row is not None
is_trait_available = trait_row is not None

# Use the function to validate and save cohort info
initial_validation = validate_and_save_cohort_info(
    is_final=False, 
    cohort=cohort, 
    info_path=json_path, 
    is_gene_available=is_gene_available, 
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
if trait_row is not None:
    # Assuming clinical_data was defined in a previous step
    try:
        # Create clinical features dataframe using the library function
        clinical_features_df = geo_select_clinical_features(
            clinical_data,
            trait=trait,
            trait_row=trait_row,
            convert_trait=convert_trait,
            age_row=age_row,
            convert_age=convert_age,
            gender_row=gender_row,
            convert_gender=convert_gender
        )
        
        # Preview the resulting dataframe
        preview = preview_df(clinical_features_df)
        print("Clinical features preview:")
        print(preview)
        
        # Ensure output directory exists
        os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
        
        # Save the clinical data to CSV
        clinical_features_df.to_csv(out_clinical_data_file)
        print(f"Clinical data saved to {out_clinical_data_file}")
    except NameError:
        print("Clinical data not defined in previous steps. Unable to extract clinical features.")
else:
    print("Clinical data not available. Skipping clinical feature extraction.")


Clinical features preview:
{'GSM3398477': [0.0], 'GSM3398478': [0.0], 'GSM3398479': [0.0], 'GSM3398480': [0.0], 'GSM3398481': [0.0], 'GSM3398482': [0.0], 'GSM3398483': [0.0], 'GSM3398484': [0.0], 'GSM3398485': [0.0], 'GSM3398486': [0.0], 'GSM3398487': [1.0], 'GSM3398488': [1.0], 'GSM3398489': [1.0], 'GSM3398490': [1.0], 'GSM3398491': [1.0], 'GSM3398492': [1.0], 'GSM3398493': [1.0], 'GSM3398494': [1.0], 'GSM3398495': [1.0], 'GSM3398496': [1.0], 'GSM3398497': [nan], 'GSM3398498': [nan], 'GSM3398499': [nan], 'GSM3398500': [nan], 'GSM3398501': [nan], 'GSM3398502': [nan], 'GSM3398503': [nan], 'GSM3398504': [nan], 'GSM3398505': [nan], 'GSM3398506': [nan]}
Clinical data saved to ../../output/preprocess/Schizophrenia/clinical_data/GSE120342.csv


### Step 3: Gene Data Extraction

In [4]:
# 1. Get the SOFT and matrix file paths again 
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
print(f"Matrix file found: {matrix_file}")

# 2. Use the get_genetic_data function from the library to get the gene_data
try:
    gene_data = get_genetic_data(matrix_file)
    print(f"Gene data shape: {gene_data.shape}")
    
    # 3. Print the first 20 row IDs (gene or probe identifiers)
    print("First 20 gene/probe identifiers:")
    print(gene_data.index[:20])
except Exception as e:
    print(f"Error extracting gene data: {e}")


Matrix file found: ../../input/GEO/Schizophrenia/GSE120342/GSE120342-GPL16311_series_matrix.txt.gz
Gene data shape: (19070, 30)
First 20 gene/probe identifiers:
Index(['100009676_at', '10000_at', '10001_at', '10002_at', '10003_at',
       '100048912_at', '100049716_at', '10004_at', '10005_at', '10006_at',
       '10007_at', '10008_at', '100093630_at', '10009_at', '1000_at',
       '100101467_at', '100101938_at', '10010_at', '100113407_at', '10011_at'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# These identifiers with the format 'cg#####' are DNA methylation probe IDs used on methylation arrays
# (likely from Illumina arrays), not gene symbols. They need to be mapped to corresponding genes.

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Analyze the gene annotation dataframe to identify which columns contain the gene identifiers and gene symbols
print("\nGene annotation preview:")
print(f"Columns in gene annotation: {gene_annotation.columns.tolist()}")
print(preview_df(gene_annotation, n=5))

# Check for gene information in the SPOT_ID.1 column which appears to contain gene names
print("\nAnalyzing SPOT_ID.1 column for gene symbols:")
if 'SPOT_ID.1' in gene_annotation.columns:
    # Extract a few sample values
    sample_values = gene_annotation['SPOT_ID.1'].head(3).tolist()
    for i, value in enumerate(sample_values):
        print(f"Sample {i+1} excerpt: {value[:200]}...")  # Print first 200 chars
        # Test the extract_human_gene_symbols function on these values
        symbols = extract_human_gene_symbols(value)
        print(f"  Extracted gene symbols: {symbols}")

# Try to find the probe IDs in the gene annotation
gene_data_id_prefix = gene_data.index[0].split('_')[0]  # Get prefix of first gene ID
print(f"\nGene data ID prefix: {gene_data_id_prefix}")

# Look for columns that might match the gene data IDs
for col in gene_annotation.columns:
    if gene_annotation[col].astype(str).str.contains(gene_data_id_prefix).any():
        print(f"Column '{col}' contains values matching gene data ID pattern")

# Check if there's any column that might contain transcript or gene IDs
print("\nChecking for columns containing transcript or gene related terms:")
for col in gene_annotation.columns:
    if any(term in col.upper() for term in ['GENE', 'TRANSCRIPT', 'SYMBOL', 'NAME', 'DESCRIPTION']):
        print(f"Column '{col}' may contain gene-related information")
        # Show sample values
        print(f"Sample values: {gene_annotation[col].head(3).tolist()}")



Gene annotation preview:
Columns in gene annotation: ['ID', 'Name', 'IlmnStrand', 'AddressA_ID', 'AlleleA_ProbeSeq', 'AddressB_ID', 'AlleleB_ProbeSeq', 'GenomeBuild', 'Chr', 'MapInfo', 'Ploidy', 'Species', 'Source', 'SourceVersion', 'SourceStrand', 'SourceSeq', 'TopGenomicSeq', 'Next_Base', 'Color_Channel', 'TSS_Coordinate', 'Gene_Strand', 'Gene_ID', 'Symbol', 'Synonym', 'Accession', 'GID', 'Annotation', 'Product', 'Distance_to_TSS', 'CPG_ISLAND', 'CPG_ISLAND_LOCATIONS', 'MIR_CPG_ISLAND', 'RANGE_GB', 'RANGE_START', 'RANGE_END', 'RANGE_STRAND', 'GB_ACC', 'ORF']
{'ID': ['cg00000292', 'cg00002426', 'cg00003994', 'cg00005847', 'cg00006414'], 'Name': ['cg00000292', 'cg00002426', 'cg00003994', 'cg00005847', 'cg00006414'], 'IlmnStrand': ['TOP', 'TOP', 'TOP', 'BOT', 'BOT'], 'AddressA_ID': [990370.0, 6580397.0, 7150184.0, 4850717.0, 6980731.0], 'AlleleA_ProbeSeq': ['AAACATTAATTACCAACCACTCTTCCAAAAAACACTTACCATTAAAACCA', 'AATATAATAACATTACCTTACCCATCTTATAATCAAACCAAACAAAAACA', 'AATAATAATAATACCCCCTAT

Column 'Name' contains values matching gene data ID pattern
Column 'IlmnStrand' contains values matching gene data ID pattern



Checking for columns containing transcript or gene related terms:
Column 'Name' may contain gene-related information
Sample values: ['cg00000292', 'cg00002426', 'cg00003994']
Column 'Gene_Strand' may contain gene-related information
Sample values: ['+', '+', '-']
Column 'Gene_ID' may contain gene-related information
Sample values: ['GeneID:487', 'GeneID:7871', 'GeneID:4223']
Column 'Symbol' may contain gene-related information
Sample values: ['ATP2A1', 'SLMAP', 'MEOX2']


### Step 6: Gene Identifier Mapping

In [7]:
# 1. Identify the columns in gene_annotation for probe IDs and gene symbols
# From the previous output, we can see:
# - 'ID' column contains the probe identifiers (cg#####) that match the gene_data index
# - 'Symbol' column contains the human gene symbols (ATP2A1, SLMAP, etc.)

# 2. Get a gene mapping dataframe with the ID and Symbol columns
gene_mapping = get_gene_mapping(gene_annotation, prob_col='ID', gene_col='Symbol')
print(f"Gene mapping dataframe shape: {gene_mapping.shape}")
print("Gene mapping preview:")
print(gene_mapping.head())

# 3. Apply the gene mapping to convert probe-level measurements to gene expression
# Use the library function to handle the many-to-many relationship between probes and genes
gene_data = apply_gene_mapping(gene_data, gene_mapping)
print(f"Transformed gene expression data shape: {gene_data.shape}")
print("First few genes and samples:")
print(gene_data.head(5).iloc[:, :5])  # Show first 5 genes and 5 samples

# Check how many genes we have after mapping
print(f"Number of genes after mapping: {len(gene_data)}")


Gene mapping dataframe shape: (27551, 2)
Gene mapping preview:
           ID    Gene
0  cg00000292  ATP2A1
1  cg00002426   SLMAP
2  cg00003994   MEOX2
3  cg00005847   HOXD3
4  cg00006414  ZNF398
Transformed gene expression data shape: (0, 30)
First few genes and samples:
Empty DataFrame
Columns: [GSM3398477, GSM3398478, GSM3398479, GSM3398480, GSM3398481]
Index: []
Number of genes after mapping: 0


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols in the gene expression data - already done in previous step
print(f"Gene data shape after normalization: {gene_data.shape}")

# Save the normalized gene data to file - already done in previous step
print(f"Normalized gene expression data saved to {out_gene_data_file}")

# 2. Link the clinical and genetic data
# Load the clinical features from the previous step
selected_clinical_df = geo_select_clinical_features(
    clinical_df=clinical_data,
    trait=trait,
    trait_row=trait_row,
    convert_trait=convert_trait,
    age_row=age_row,
    convert_age=convert_age,
    gender_row=gender_row,
    convert_gender=convert_gender
)

print(f"Selected clinical data shape: {selected_clinical_df.shape}")
print("Clinical data preview:")
print(selected_clinical_df.head())

# Link the clinical and genetic data directly using the properly encoded clinical data
linked_data = geo_link_clinical_genetic_data(selected_clinical_df, gene_data)
print(f"Linked data shape: {linked_data.shape}")
print("Linked data preview (first 5 rows, 5 columns):")
print(linked_data.iloc[:5, :5])

# 3. Handle missing values
linked_data = handle_missing_values(linked_data, trait)
print(f"Data shape after handling missing values: {linked_data.shape}")

# 4. Check for bias in features
is_biased, linked_data = judge_and_remove_biased_features(linked_data, trait)
print(f"Data shape after removing biased features: {linked_data.shape}")

# 5. Validate and save cohort information
is_usable = validate_and_save_cohort_info(
    is_final=True,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=True,
    is_trait_available=True,
    is_biased=is_biased,
    df=linked_data,
    note="Dataset contains gene expression data related to alcohol dependence but was evaluated for Schizophrenia."
)

# 6. Save the linked data if usable
if is_usable:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    linked_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print("Dataset is not usable for analysis. No linked data file saved.")

Gene data shape after normalization: (0, 30)
Normalized gene expression data saved to ../../output/preprocess/Schizophrenia/gene_data/GSE120342.csv
Selected clinical data shape: (1, 30)
Clinical data preview:
               GSM3398477  GSM3398478  GSM3398479  GSM3398480  GSM3398481  \
Schizophrenia         0.0         0.0         0.0         0.0         0.0   

               GSM3398482  GSM3398483  GSM3398484  GSM3398485  GSM3398486  \
Schizophrenia         0.0         0.0         0.0         0.0         0.0   

               ...  GSM3398497  GSM3398498  GSM3398499  GSM3398500  \
Schizophrenia  ...         NaN         NaN         NaN         NaN   

               GSM3398501  GSM3398502  GSM3398503  GSM3398504  GSM3398505  \
Schizophrenia         NaN         NaN         NaN         NaN         NaN   

               GSM3398506  
Schizophrenia         NaN  

[1 rows x 30 columns]
Linked data shape: (30, 1)
Linked data preview (first 5 rows, 5 columns):
           Schizophrenia
GSM3398

/media/techt/DATA/GenoAgent/tools/preprocess.py:400: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  linked_data = pd.concat([clinical_df, genetic_df], axis=0).T
